In [1]:
import pandas as pd
import pandas_ta as ta
from sqlalchemy import create_engine

print(f"Pandas Version: {pd.__version__}")
print(f"Pandas TA Version: {ta.version}")

Pandas Version: 2.3.3
Pandas TA Version: 0.4.71b0


In [2]:
# Define o nome do arquivo do banco de dados
db_file = "dados_acoes.db"

# Cria a "engine" de conexão do SQLAlchemy
# O prefixo 'sqlite:///' informa que é um arquivo SQLite
engine = create_engine(f'sqlite:///{db_file}')

try:
    # Carrega cada tabela em um DataFrame do Pandas
    # O Pandas usará a 'engine' para ler os dados
    candles_diarios = pd.read_sql("candles_diarios", engine)
    candles_semanais = pd.read_sql("candles_semanais", engine)
    candles_mensais = pd.read_sql("candles_mensais", engine)

    print("Tabelas carregadas com sucesso do banco de dados!")
    print(f"Total de registros 'candles_diarios': {len(candles_diarios)}")
    print(f"Total de registros 'candles_semanais': {len(candles_semanais)}")
    print(f"Total de registros 'candles_mensais': {len(candles_mensais)}")

except Exception as e:
    print(f"Erro ao ler o banco de dados: {e}")
    print("Verifique se o arquivo 'dados_acoes.db' está na mesma pasta do seu Jupyter Notebook.")

Tabelas carregadas com sucesso do banco de dados!
Total de registros 'candles_diarios': 298956
Total de registros 'candles_semanais': 63758
Total de registros 'candles_mensais': 14695


In [3]:
# --- AJUSTE ESTE MAPA CUIDADOSAMENTE ---
mapa_colunas = {
    'Data': 'datetime',      
    'Abertura': 'open',  
    'Maxima': 'high',    
    'Minima': 'low',     
    'Fechamento': 'close', 
    'Volume': 'volume'   
}

try:
    # Renomeia colunas, criando NOVOS dataframes "processados"
    diarios_proc = candles_diarios.rename(columns=mapa_colunas)
    semanais_proc = candles_semanais.rename(columns=mapa_colunas)
    mensais_proc = candles_mensais.rename(columns=mapa_colunas)

    print("DataFrames processados (clones) foram criados após renomeação.")
    
    # Ordena os dataframes processados (isso também cria cópias)
    if 'date' in diarios_proc.columns:
        diarios_proc = diarios_proc.sort_values(by='date')
        semanais_proc = semanais_proc.sort_values(by='date')
        mensais_proc = mensais_proc.sort_values(by='date')
        print("DataFrames processados foram ordenados por data.")

    print("\n--- Verificação ---")
    print(f"Colunas Originais (ex: diarios): {candles_diarios.columns.to_list()}")
    print(f"Colunas Processadas (ex: diarios_proc): {diarios_proc.columns.to_list()}")

except Exception as e:
    print(f"Erro ao renomear colunas: {e}")
    print("Verifique se os nomes no 'mapa_colunas' (lado esquerdo) estão corretos.")

DataFrames processados (clones) foram criados após renomeação.

--- Verificação ---
Colunas Originais (ex: diarios): ['datetime', 'close', 'high', 'low', 'open', 'volume', 'ticker']
Colunas Processadas (ex: diarios_proc): ['datetime', 'close', 'high', 'low', 'open', 'volume', 'ticker']


In [4]:
diarios_proc

,datetime,close,high,low,open,volume,ticker
0,2000-01-03,1.156394,1.156394,1.156394,1.156394,35389440000,PETR4.SA
1,2000-01-04,1.092423,1.092423,1.092423,1.092423,28861440000,PETR4.SA
2,2000-01-05,1.081401,1.081401,1.081401,1.081401,43033600000,PETR4.SA
3,2000-01-06,1.077661,1.077661,1.077661,1.077661,34055680000,PETR4.SA
4,2000-01-07,1.082582,1.082582,1.082582,1.082582,20912640000,PETR4.SA
...,...,...,...,...,...,...,...
298951,2025-10-20,4.460000,4.750000,4.450000,4.600000,2684400,CASH3.SA
298952,2025-10-21,4.470000,4.510000,4.400000,4.480000,1218400,CASH3.SA
298953,2025-10-22,4.340000,4.470000,4.290000,4.450000,1850000,CASH3.SA
298954,2025-10-23,4.470000,4.490000,4.350000,4.350000,1581200,CASH3.SA


In [5]:
def calcular_indicadores_tecnicos(df_original):
    """
    Calcula um conjunto de indicadores técnicos usando pandas_ta 
    e os anexa a uma cópia do DataFrame.
    """
    # Cria uma cópia para não alterar o DataFrame de entrada
    df = df_original.copy()
    
    # 1. Média Móvel Exponencial (EMA) de 9 períodos
    df.ta.ema(length=9, append=True)
    
    # 2. Média Móvel Simples (SMA) de 21 períodos
    df.ta.sma(length=21, append=True)
    
    # 3. Média Móvel Simples (SMA) de 50 períodos
    df.ta.sma(length=50, append=True)
    
    # 4. Média Móvel Simples (SMA) de 200 períodos
    df.ta.sma(length=200, append=True)
    
    # 5. RSI (Índice de Força Relativa)
    df.ta.rsi(length=14, append=True)
    
    # 6. MACD
    df.ta.macd(append=True)
    
    # 7. Bandas de Bollinger (BBANDS)
    df.ta.bbands(length=20, append=True)
    
    # 8. Oscilador Estocástico (STOCH)
    df.ta.stoch(append=True)
    
    # 9. OBV (On-Balance Volume)
    df.ta.obv(append=True)
    
    # 10. ATR (Average True Range)
    df.ta.atr(length=14, append=True)
    
    # 11. ADX (Average Directional Index)
    df.ta.adx(length=14, append=True)
    
    return df

print("Função 'calcular_indicadores_tecnicos' definida.")

Função 'calcular_indicadores_tecnicos' definida.


In [6]:
# Verifica se os DataFrames processados existem antes de calcular
if 'diarios_proc' in locals():
    # Os DataFrames com os cálculos finais
    df_diario_calculado = calcular_indicadores_tecnicos(diarios_proc)
    df_semanal_calculado = calcular_indicadores_tecnicos(semanais_proc)
    df_mensal_calculado = calcular_indicadores_tecnicos(mensais_proc)

    print("Indicadores calculados para os 3 DataFrames (em clones separados).")
else:
    print("Os DataFrames processados ('diarios_proc', etc.) não foram criados. Verifique a Célula 4.")

Indicadores calculados para os 3 DataFrames (em clones separados).


In [7]:
print("--- Resultado Final (df_diario_calculado) ---")

print(df_diario_calculado.tail())

--- Resultado Final (df_diario_calculado) ---
         datetime  close  high   low  open   volume    ticker     EMA_9  \
298951 2025-10-20   4.46  4.75  4.45  4.60  2684400  CASH3.SA  4.423796   
298952 2025-10-21   4.47  4.51  4.40  4.48  1218400  CASH3.SA  4.433037   
298953 2025-10-22   4.34  4.47  4.29  4.45  1850000  CASH3.SA  4.414430   
298954 2025-10-23   4.47  4.49  4.35  4.35  1581200  CASH3.SA  4.425544   
298955 2025-10-24   4.65  4.76  4.45  4.45  2732900  CASH3.SA  4.470435   

          SMA_21  SMA_50  ...  BBP_20_2.0_2.0  STOCHk_14_3_3  STOCHd_14_3_3  \
298951  4.240476  4.5882  ...        0.791438      77.118564      81.160500   
298952  4.258095  4.5662  ...        0.775300      72.963620      78.414552   
298953  4.268571  4.5364  ...        0.588597      58.728630      69.603605   
298954  4.283810  4.5138  ...        0.748682      56.452979      62.715076   
298955  4.319048  4.4984  ...        0.964184      62.564096      59.248568   

        STOCHh_14_3_3       

In [8]:
try:
    if 'df_diario_calculado' in locals():
        # Define os nomes das novas tabelas
        tabela_diario_final = "diario_com_indicadores"
        tabela_semanal_final = "semanal_com_indicadores"
        tabela_mensal_final = "mensal_com_indicadores"

        # Salva os DataFrames no banco de dados
        # if_exists='replace': Substitui a tabela se ela já existir de uma execução anterior
        # index=False: Não salva o índice do Pandas como uma coluna no DB
        
        df_diario_calculado.to_sql(tabela_diario_final, con=engine, if_exists='replace', index=False)
        print(f"DataFrame diário salvo na tabela: '{tabela_diario_final}'")
        
        df_semanal_calculado.to_sql(tabela_semanal_final, con=engine, if_exists='replace', index=False)
        print(f"DataFrame semanal salvo na tabela: '{tabela_semanal_final}'")
        
        df_mensal_calculado.to_sql(tabela_mensal_final, con=engine, if_exists='replace', index=False)
        print(f"DataFrame mensal salvo na tabela: '{tabela_mensal_final}'")
        
        print("\nSucesso! Os dados calculados foram salvos em novas tabelas no arquivo 'dados_acoes.db'.")
        
    else:
        print("Os DataFrames calculados não foram encontrados. Pulei a etapa de salvar.")
        
except Exception as e:
    print(f"Erro ao salvar os dados no banco de dados: {e}")

DataFrame diário salvo na tabela: 'diario_com_indicadores'
DataFrame semanal salvo na tabela: 'semanal_com_indicadores'
DataFrame mensal salvo na tabela: 'mensal_com_indicadores'

Sucesso! Os dados calculados foram salvos em novas tabelas no arquivo 'dados_acoes.db'.
